In [1]:
from random import random, sample

from threading import Thread

from time import sleep

# Programa 1

In [2]:
class Tortuga(Thread):
    def __init__(self,meta):
        self.meta = meta
        self.casilla_actual = 1
        super().__init__()
    
    def __str__(self):
        p = f"posicion-{self.casilla_actual}"
        return p + self.casilla_actual*" " + "T\n"
    
    def determinar_suceso(self):
        probabilidad = random()
        if probabilidad <= 0.5:
            self.avanzar_rapido()
        elif 0.5 < probabilidad <= 0.7:
            self.resbalar()
        else:
            # probabilidad >= 0.2
            self.avanzar_lento()
    
    def avanzar_rapido(self):
        self.casilla_actual += 3
    
    def resbalar(self):
        self.casilla_actual = max(1,self.casilla_actual-6)
    
    def avanzar_lento(self):
        self.casilla_actual += 1
    
    def ha_ganado(self):
        if self.casilla_actual >= 70:
            self.meta.pop()
            print("La tortuga llegó a la meta")
            return True
        return False
    
    def run(self):
        while self.meta:
            sleep(1)
            self.determinar_suceso()
            print(self)
            meta_alcanzada = self.ha_ganado()
            

In [3]:
class Liebre(Thread):
    def __init__(self,meta):
        self.meta = meta
        self.casilla_actual = 1
        super().__init__()
    
    def __str__(self):
        p = f"posicion-{self.casilla_actual}"
        return p + self.casilla_actual*" " + "L\n"
    
    def determinar_suceso(self):
        probabilidad = random()
        if 0 <= probabilidad <= 0.2:
            self.duerme()
        elif 0.2 < probabilidad <= 0.4:
            self.dar_gran_salto()
        elif 0.4 < probabilidad <= 0.5:
            self.dar_resbalon_grande()
        elif 0.5 < probabilidad <= 0.8:
            self.dar_pequeño_salto()
        else:
            self.dar_resbalon_pequeño()
            
    def dar_pequeño_salto(self):
        self.casilla_actual += 1
    
    def duerme(self):
        pass
    
    def dar_gran_salto(self):
        self.casilla_actual += 9
    
    def dar_resbalon_pequeño(self):
        self.casilla_actual = max(1,self.casilla_actual-2)
    
    def dar_resbalon_grande(self):
        self.casilla_actual = max(1,self.casilla_actual-12)
        
    def ha_ganado(self):
        if self.casilla_actual >= 70:
            self.meta.pop()
            print("La liebre llegó a la meta")
            return True
        return False
    
    def run(self):
        while self.meta:
            sleep(1)
            self.determinar_suceso()
            print(self)
            meta_alcanzada = self.ha_ganado()

In [4]:
meta = ['meta']
liebre = Liebre(meta)
tortuga = Tortuga(meta)

liebre.start()
tortuga.start()

if liebre.casilla_actual >= 70 and tortuga.casilla_actual >= 70:
    print("Es un empate")

posicion-1 L
posicion-4    T


posicion-1 T
posicion-10          L


posicion-1 T
posicion-1 L


posicion-1 L
posicion-2  T


posicion-2  L
posicion-3   T


posicion-4    T
posicion-1 L


posicion-2  L
posicion-7       T


posicion-1 L

posicion-8        T

posicion-1 L
posicion-9         T


posicion-3   T
posicion-10          L


posicion-10          L
posicion-6      T


posicion-10          L
posicion-1 T


posicion-4    T
posicion-10          L


posicion-1 T

posicion-10          L

posicion-4    T
posicion-19                   L


posicion-7       T

posicion-19                   L

posicion-20                    L
posicion-8        T


posicion-11           T
posicion-20                    L


posicion-5     T
posicion-29                             L


posicion-30                              L

posicion-8        T

posicion-9         T
posicion-31                               L


posicion-31                               L

posicion-3   T

posicion-32                        

# Programa 2

In [5]:
from threading import Thread
from time import time
import numpy as np

## Secuencial

In [6]:
def producto_matricial(A,B):
    rA,cA = A.shape
    rB,cB = B.shape
    C = np.zeros((rA,cB))
    if cA == rB:
        for i,r in enumerate(A):
            for j,c in enumerate(B.T):
                C[i,j] = r @ c
    return C

In [7]:
t0 = time()
A = np.random.random((400,400))
B = np.random.random((400,400))
C = producto_matricial(A,B)
print(f"Tiempo secuencial: {time()-t0}")

Tiempo secuencial: 0.5254008769989014


# Concurrente

In [8]:
def producto_matricial(A,B,sx,sy):
    global C
    n,m = C.shape
    rango_x = range(sx*int(np.floor(n/2)), (sx+1)*int(np.ceil(n/2))-1)
    rango_y = range(sy*int(np.floor(m/2)), (sy+1)*int(np.ceil(m/2))-1)
    n,m = A.shape
    m,p = B.shape
    for i in rango_x:
        for j in rango_y:
            suma = 0
            for k in range(m):
                suma += A[i,k]*B[k,j]
            C[i,j] = suma

In [9]:
A = np.random.random((400,400))
B = np.random.random((400,400))

C = np.zeros((A.shape[0],B.shape[1]))

p1 = Thread(target=producto_matricial, args=(A,B,0,0))
p2 = Thread(target=producto_matricial, args=(A,B,0,1))
p3 = Thread(target=producto_matricial, args=(A,B,1,0))
p4 = Thread(target=producto_matricial, args=(A,B,1,1))

t0 = time()

p1.start()
p2.start()
p3.start()
p4.start()

p1.join()
p2.join()
p3.join()
p4.join()

print(f"Tiempo secuencial: {time()-t0}")

Tiempo secuencial: 43.86326599121094
